In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import warnings

class DynamicModeDecomposition:
    def __init__(self, rank=None, svd_rank=None):
        """
        動的モード分解（DMD）クラス
        論文 "Dynamic mode decomposition of numerical and experimental data" に基づく実装
        
        Parameters:
        rank: DMDで使用するランク（Noneの場合は自動決定）
        svd_rank: SVDで使用するランク（Noneの場合は自動決定）
        """
        self.rank = rank
        self.svd_rank = svd_rank
        self.modes = None
        self.eigenvalues = None
        self.amplitudes = None
        self.Phi = None  # DMDモード
        self.omega = None  # 連続時間固有値
        self.b = None  # 初期振幅
        self.A_tilde = None  # 低次元線形演算子
        
    def fit(self, X, dt=1.0):
        """
        DMDアルゴリズムを実行
        
        Parameters:
        X: データ行列 (n_features x n_timesteps)
        dt: 時間刻み
        """
        # Step 1: データをX1とX2に分割
        X1 = X[:, :-1]  # x_1, x_2, ..., x_{m-1}
        X2 = X[:, 1:]   # x_2, x_3, ..., x_m
        
        # Step 2: X1のSVD分解
        U, s, Vt = svd(X1, full_matrices=False)
        
        # Step 3: ランクの決定とtruncation 
        if self.svd_rank is None:
            # 特異値の閾値による自動ランク決定
            tolerance = 1e-10
            self.svd_rank = np.sum(s > tolerance)
        
        r = min(self.svd_rank, len(s))
        U_r = U[:, :r]
        s_r = s[:r]
        V_r = Vt[:r, :].T
        
        # Step 4: 低次元線形演算子A_tildeの計算
        #self.A_tilde = U_r.T @ X2 @ V_r @ np.diag(1/s_r)
        self.A_tilde = U_r.T @ X2 @ V_r @ np.linalg.pinv(s_r)
        
        # Step 5: A_tildeの固有値分解
        eigenvals, W = np.linalg.eig(self.A_tilde)
        
        # Step 6: DMDモードの計算
        #self.Phi = X2 @ V_r @ np.diag(1/s_r) @ W
        self.Phi = X2 @ V_r @ np.linalg.pinv(s_r) @ W
        
        # 離散時間固有値
        self.eigenvalues = eigenvals
        
        # 連続時間固有値（論文の式に基づく）
        #self.omega = np.log(eigenvals) / dt
        self.omega = eigenvals #dt=1の場合 expがpredictのところで入れていないためlog入れていない
        
        # Step 7: 初期振幅の計算
        self.b = np.linalg.pinv(self.Phi) @ X[:, 0]
        
        # 使用するモード数の決定
        if self.rank is None:
            self.rank = r
        else:
            self.rank = min(self.rank, r)
            
        return self
    
    def predict(self, t_eval, x0=None):
        """
        指定された時刻での状態を予測
        
        Parameters:
        t_eval: 評価時刻
        x0: 初期条件（Noneの場合は学習データの初期条件を使用）
        
        Returns:
        X_pred: 予測された状態 (n_features )
        """
        if self.Phi is None:
            raise ValueError("まずfit()を実行してください")
        
        if x0 is not None:
            # 新しい初期条件での振幅計算
            b = np.linalg.pinv(self.Phi) @ x0
        else:
            b = self.b
        
        # 時間発展の計算
        # x(t) = Φ * diag(b) * exp(ω*t)
        
        time_evolution = np.array(self.omega[:self.rank, np.newaxis] **t_eval)
        
        # 予測
        X_pred = self.Phi[:, :self.rank] @ (b[:self.rank, np.newaxis] * time_evolution)#ここ直すべき
        
        return X_pred
    
    def compute_dmd_solution(self, t_eval, mode_indices=None):
        """
        特定のモードのみを使用したDMD解を計算
        
        Parameters:
        t_eval: 評価時刻
        mode_indices: 使用するモードのインデックス（Noneの場合は全モード）
        """
        if mode_indices is None:
            mode_indices = range(self.rank)
        
        time_evolution = np.exp(np.outer(self.omega[mode_indices], t_eval))
        X_dmd = self.Phi[:, mode_indices] @ (self.b[mode_indices, np.newaxis] * time_evolution)
        
        return X_dmd

In [ ]:
# テストデータの生成
X, t, dt = create_test_data()

print("データ形状:", X.shape)
print("時間刻み:", dt)

# DMDの実行
dmd = DynamicModeDecomposition(rank=10, svd_rank=15)
dmd.fit(X, dt=dt)

# 予測
X_pred = dmd.predict(t)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import warnings


In [2]:
def create_test_data():
    """テスト用のデータを生成"""
    # 時間軸
    t = np.linspace(0, 10, 200)
    dt = t[1] - t[0]
    
    # 複数の動的モードを持つシステム
    # モード1: 減衰振動
    mode1 = np.exp(-0.1 * t) * np.sin(2 * np.pi * 0.5 * t)
    # モード2: 成長振動
    mode2 = np.exp(0.05 * t) * np.cos(2 * np.pi * 0.8 * t)
    # モード3: 純粋減衰
    mode3 = np.exp(-0.2 * t)
    
    # 空間構造を追加
    n_spatial = 50
    x_spatial = np.linspace(0, 1, n_spatial)
    
    # 各モードに異なる空間構造を与える
    spatial1 = np.sin(np.pi * x_spatial)
    spatial2 = np.sin(2 * np.pi * x_spatial)
    spatial3 = np.exp(-((x_spatial - 0.5) / 0.2)**2)  # ガウシアン
    
    # データ行列の構築
    X = (spatial1[:, np.newaxis] @ mode1[np.newaxis, :] + 
         spatial2[:, np.newaxis] @ mode2[np.newaxis, :] + 
         spatial3[:, np.newaxis] @ mode3[np.newaxis, :])
    
    # ノイズを追加
    noise_level = 0.05
    X += noise_level * np.random.randn(*X.shape)
    
    return X, t, dt

In [8]:
X, t, dt = create_test_data()
print("データ形状:", X.shape)
#print("時間刻み:", dt)
print(X)

データ形状: (50, 200)
[[-0.0745888   0.05321973  0.01286006 ...  0.0273461   0.02968766
   0.0828189 ]
 [ 0.23515432  0.09822955  0.04070709 ...  0.13957109  0.24122071
   0.15837971]
 [ 0.28280217  0.21683989  0.28078008 ...  0.30791642  0.44357456
   0.44619883]
 ...
 [-0.30979545 -0.22805543 -0.18186889 ... -0.36120284 -0.48299688
  -0.38423087]
 [-0.193734   -0.13267655 -0.13453994 ... -0.10046946 -0.27859123
  -0.23440034]
 [ 0.00326939  0.10574404  0.02580776 ... -0.09704743 -0.0419091
  -0.01568088]]


In [17]:
a=np.array([[1],[2],[3]])
b=np.array([[2],[3],[4]])
c =a*b
print(c)

[[ 2]
 [ 6]
 [12]]


In [15]:
d = np.array(a**2)
d

array([[1],
       [4],
       [9]])

In [16]:
print(np.exp(a))

[[ 2.71828183]
 [ 7.3890561 ]
 [20.08553692]]
